In [1]:
import numpy as np
import pandas as pd

## Data Loading

- Load Netflix dataset

In [2]:
netflix_df=pd.read_csv('netflix_titles.csv', usecols=['title','type','director','country','release_year','rating','duration','listed_in'])

In [3]:
netflix_df.head()

,type,title,director,country,release_year,rating,duration,listed_in
0,Movie,Dick Johnson Is Dead,Kirsten Johnson,United States,2020,PG-13,90 min,Documentaries
1,TV Show,Blood & Water,NaN,South Africa,2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries"
2,TV Show,Ganglands,Julien Leclercq,NaN,2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act..."
3,TV Show,Jailbirds New Orleans,NaN,NaN,2021,TV-MA,1 Season,"Docuseries, Reality TV"
4,TV Show,Kota Factory,NaN,India,2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ..."


- IMDb basics
- Since the dataset is large (contains more than 10m records), loading it entirely is inefficient. To efficiently explore the data, we will initially load only a subset of rows using nrows and select only the necessary columns.

In [4]:
url = "https://datasets.imdbws.com/"
imdb_basics_df_temp = pd.read_csv(url+"title.basics.tsv.gz", sep='\t', compression='gzip', na_values='\\N', nrows=50)
imdb_basics_df_temp.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,NaN,1.0,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,NaN,5.0,"Animation,Short"
2,tt0000003,short,Poor Pierrot,Pauvre Pierrot,0,1892,NaN,5.0,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,NaN,12.0,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,NaN,1.0,Short


In [5]:
imdb_basics_df= pd.read_csv(url+"title.basics.tsv.gz", sep='\t', compression='gzip', na_values='\\N', usecols=['tconst','titleType','primaryTitle','isAdult','startYear'])

In [6]:
imdb_basics_df= imdb_basics_df[imdb_basics_df['titleType'].isin(['movie','tvMovie','tvSeries','tvMiniSeries'])]

In [7]:
imdb_ratings_df=pd.read_csv(url+"title.ratings.tsv.gz", sep='\t', compression='gzip', na_values='\\N')

In [8]:
imdb_ratings_df.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,2182
1,tt0000002,5.5,302
2,tt0000003,6.4,2256
3,tt0000004,5.2,194
4,tt0000005,6.2,2994


## Data Preprocessing

#### Basic Information about all the datasets

In [9]:
def basic_information(*dfs):
    for i, df in enumerate(dfs, 1):
        print("\n" + "-"*30 + f" DataFrame {i} " + "-"*30 + "\n")
        
        print("Basic Info:")
        print(df.info())
        
        print("\nDuplicates Check:")
        print(f"Number of duplicate rows: {df.duplicated().sum()}")
        
        print("\nNull Value Check:")
        print(df.isna().sum())
        
        print("\nUnique Values Check:")
        print(df.nunique())
        print("\n" + "="*70 + "\n")


In [10]:
basic_information(netflix_df,imdb_basics_df,imdb_ratings_df)


------------------------------ DataFrame 1 ------------------------------

Basic Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   type          8807 non-null   object
 1   title         8807 non-null   object
 2   director      6173 non-null   object
 3   country       7976 non-null   object
 4   release_year  8807 non-null   int64 
 5   rating        8803 non-null   object
 6   duration      8804 non-null   object
 7   listed_in     8807 non-null   object
dtypes: int64(1), object(7)
memory usage: 550.6+ KB
None

Duplicates Check:
Number of duplicate rows: 0

Null Value Check:
type               0
title              0
director        2634
country          831
release_year       0
rating             4
duration           3
listed_in          0
dtype: int64

Unique Values Check:
type               2
title           8807
director        45

- From the above result we could see that there are null values for Netflix data and IMDb bascis data. No duplicates found
for all the datasets
- We could see that the type for "start year" is float64, lets convert it to Int64 as it is not practical to contain
decimal values for any year.
- Also by observing the Netflix data, we could see the below observations
    - "Duration"1 is considered as string value as it contains "min" and "seasons" for movie and TV show categories respectively. So, lets make the Duration column into two and convert it to Integer data type
    - "Director", "Country", "listed_in" columns contain multiple values for a single record seperated by comma. So, lets explode them to ensure each record contains only single value.

### Let's view the unique values of few colums

In [10]:
def view_unique_values(*dfs, max_values=20):
    
    for i, df in enumerate(dfs, 1):
        print(f"\n{'='*40}\nDataFrame {i}\n{'='*40}")
        
        for col in df.columns:
            unique_count = df[col].nunique(dropna=False)
            print(f"\n{'-'*30}\nColumn: {col}\nUnique Count: {unique_count}")
            
            unique_vals = df[col].unique()
            if unique_count <= max_values:
                print("Unique Values:", unique_vals)
            elif unique_count <= 150:
                print("Sample Unique Values:", unique_vals[:max_values], "...")
            


In [12]:
view_unique_values(netflix_df,imdb_basics_df)


DataFrame 1

------------------------------
Column: type
Unique Count: 2
Unique Values: ['Movie' 'TV Show']

------------------------------
Column: title
Unique Count: 8807

------------------------------
Column: director
Unique Count: 4529

------------------------------
Column: country
Unique Count: 749

------------------------------
Column: release_year
Unique Count: 74
Sample Unique Values: [2020 2021 1993 2018 1996 1998 1997 2010 2013 2017 1975 1978 1983 1987
 2012 2001 2014 2002 2003 2004] ...

------------------------------
Column: rating
Unique Count: 18
Unique Values: ['PG-13' 'TV-MA' 'PG' 'TV-14' 'TV-PG' 'TV-Y' 'TV-Y7' 'R' 'TV-G' 'G'
 'NC-17' '74 min' '84 min' '66 min' 'NR' nan 'TV-Y7-FV' 'UR']

------------------------------
Column: duration
Unique Count: 221

------------------------------
Column: listed_in
Unique Count: 514

DataFrame 2

------------------------------
Column: tconst
Unique Count: 1236351

------------------------------
Column: titleType
Unique Count: 4
U

#### Function to convert the data type

In [11]:
def data_type_conversion(df, datatype, *columns):
    for col in columns:
        df[col] = df[col].astype(datatype)


#### Converting "start year" column to integer

In [12]:
data_type_conversion(imdb_basics_df,'Int64','startYear')
imdb_basics_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1236351 entries, 8 to 11981257
Data columns (total 5 columns):
 #   Column        Non-Null Count    Dtype 
---  ------        --------------    ----- 
 0   tconst        1236351 non-null  object
 1   titleType     1236351 non-null  object
 2   primaryTitle  1236350 non-null  object
 3   isAdult       1236351 non-null  int64 
 4   startYear     1095858 non-null  Int64 
dtypes: Int64(1), int64(1), object(3)
memory usage: 57.8+ MB


#### Transforming the "duration" column and changing the data type to Integer

In [13]:
# Extract numeric part from duration
netflix_df['duration_num'] = netflix_df['duration'].str.extract('(\d+)').astype(float)

# Separating the columns
netflix_df['Movie_duration'] = np.where(netflix_df['type']=='Movie', netflix_df['duration_num'], np.nan)
netflix_df['TV Show_duration'] = np.where(netflix_df['type']=='TV Show', netflix_df['duration_num'], np.nan)

# Drop the old column
netflix_df.drop(columns=['duration', 'duration_num'], inplace=True)

# Converting the datatype to Integer
data_type_conversion(netflix_df,'Int64','Movie_duration','TV Show_duration')

#### Verify the results

In [14]:
netflix_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   type              8807 non-null   object
 1   title             8807 non-null   object
 2   director          6173 non-null   object
 3   country           7976 non-null   object
 4   release_year      8807 non-null   int64 
 5   rating            8803 non-null   object
 6   listed_in         8807 non-null   object
 7   Movie_duration    6128 non-null   Int64 
 8   TV Show_duration  2676 non-null   Int64 
dtypes: Int64(2), int64(1), object(6)
memory usage: 636.6+ KB


#### Exploding the columns - director, country, listed_in

In [15]:
netflix_df['director'] = netflix_df['director'].str.split(', ')
netflix_df['country'] = netflix_df['country'].str.split(', ')
netflix_df['listed_in'] = netflix_df['listed_in'].str.split(', ')
netflix_df = netflix_df.explode('director').explode('country').explode('listed_in').reset_index(drop=True)

#### Verify the results

In [16]:
print(netflix_df['director'].unique())
print()
print(netflix_df['country'].unique())
print()
print(netflix_df['listed_in'].unique())

['Kirsten Johnson' nan 'Julien Leclercq' ... 'Majid Al Ansari'
 'Peter Hewitt' 'Mozez Singh']

['United States' 'South Africa' nan 'India' 'Ghana' 'Burkina Faso'
 'United Kingdom' 'Germany' 'Ethiopia' 'Czech Republic' 'Mexico' 'Turkey'
 'Australia' 'France' 'Finland' 'China' 'Canada' 'Japan' 'Nigeria' 'Spain'
 'Belgium' 'South Korea' 'Singapore' 'Italy' 'Romania' 'Argentina'
 'Venezuela' 'Hong Kong' 'Russia' '' 'Ireland' 'Nepal' 'New Zealand'
 'Brazil' 'Greece' 'Jordan' 'Colombia' 'Switzerland' 'Israel' 'Taiwan'
 'Bulgaria' 'Algeria' 'Poland' 'Saudi Arabia' 'Thailand' 'Indonesia'
 'Egypt' 'Denmark' 'Kuwait' 'Netherlands' 'Malaysia' 'Vietnam' 'Hungary'
 'Sweden' 'Lebanon' 'Syria' 'Philippines' 'Iceland' 'United Arab Emirates'
 'Norway' 'Qatar' 'Mauritius' 'Austria' 'Cameroon' 'Palestine' 'Uruguay'
 'United Kingdom,' 'Kenya' 'Chile' 'Luxembourg' 'Cambodia' 'Bangladesh'
 'Portugal' 'Cayman Islands' 'Senegal' 'Serbia' 'Malta' 'Namibia' 'Angola'
 'Peru' 'Mozambique' 'Cambodia,' 'Belarus' 'Z

## Imputation

In [17]:
netflix_df['director']=netflix_df['director'].fillna("Unknown")

In [22]:
netflix_df.isna().sum()

type                    0
title                   0
director                0
country                 0
release_year            0
rating                  0
listed_in               0
Movie_duration       7093
TV Show_duration    18800
dtype: int64

In [19]:
netflix_df['country']=netflix_df['country'].fillna(netflix_df['country'].mode()[0])

In [21]:
#netflix_df['rating']=netflix_df['rating'].fillna("NR")
netflix_df['rating']=np.where(netflix_df['rating'].str.contains('min', na=False) | netflix_df['rating'].isna() , "NR",netflix_df['rating']) 

In [23]:
netflix_df.duplicated().sum()

6

In [24]:
netflix_df=netflix_df.drop_duplicates()

In [25]:
netflix_df['country'].unique()

array(['United States', 'South Africa', 'India', 'Ghana', 'Burkina Faso',
       'United Kingdom', 'Germany', 'Ethiopia', 'Czech Republic',
       'Mexico', 'Turkey', 'Australia', 'France', 'Finland', 'China',
       'Canada', 'Japan', 'Nigeria', 'Spain', 'Belgium', 'South Korea',
       'Singapore', 'Italy', 'Romania', 'Argentina', 'Venezuela',
       'Hong Kong', 'Russia', '', 'Ireland', 'Nepal', 'New Zealand',
       'Brazil', 'Greece', 'Jordan', 'Colombia', 'Switzerland', 'Israel',
       'Taiwan', 'Bulgaria', 'Algeria', 'Poland', 'Saudi Arabia',
       'Thailand', 'Indonesia', 'Egypt', 'Denmark', 'Kuwait',
       'Netherlands', 'Malaysia', 'Vietnam', 'Hungary', 'Sweden',
       'Lebanon', 'Syria', 'Philippines', 'Iceland',
       'United Arab Emirates', 'Norway', 'Qatar', 'Mauritius', 'Austria',
       'Cameroon', 'Palestine', 'Uruguay', 'United Kingdom,', 'Kenya',
       'Chile', 'Luxembourg', 'Cambodia', 'Bangladesh', 'Portugal',
       'Cayman Islands', 'Senegal', 'Serbia', 'Ma

In [27]:
imdb_basics_df['startYear']=imdb_basics_df['startYear'].fillna(-1)

In [28]:
imdb_basics_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1236351 entries, 8 to 11981257
Data columns (total 5 columns):
 #   Column        Non-Null Count    Dtype 
---  ------        --------------    ----- 
 0   tconst        1236351 non-null  object
 1   titleType     1236351 non-null  object
 2   primaryTitle  1236350 non-null  object
 3   isAdult       1236351 non-null  int64 
 4   startYear     1236351 non-null  Int64 
dtypes: Int64(1), int64(1), object(3)
memory usage: 57.8+ MB


In [29]:
netflix_imdb_df=netflix_df.merge(imdb_basics_df,left_on=['title','release_year'], right_on=['primaryTitle','startYear']).merge(imdb_ratings_df,on='tconst')

In [30]:
netflix_imdb_df.shape

(17781, 16)

In [31]:
netflix_imdb_df.head()

,type,title,director,country,release_year,rating,listed_in,Movie_duration,TV Show_duration,tconst,titleType,primaryTitle,isAdult,startYear,averageRating,numVotes
0,Movie,Dick Johnson Is Dead,Kirsten Johnson,United States,2020,PG-13,Documentaries,90,<NA>,tt11394180,movie,Dick Johnson Is Dead,0,2020,7.4,7549
1,TV Show,Ganglands,Julien Leclercq,United States,2021,TV-MA,Crime TV Shows,<NA>,1,tt13278100,tvSeries,Ganglands,0,2021,7.2,4913
2,TV Show,Ganglands,Julien Leclercq,United States,2021,TV-MA,International TV Shows,<NA>,1,tt13278100,tvSeries,Ganglands,0,2021,7.2,4913
3,TV Show,Ganglands,Julien Leclercq,United States,2021,TV-MA,TV Action & Adventure,<NA>,1,tt13278100,tvSeries,Ganglands,0,2021,7.2,4913
4,TV Show,Jailbirds New Orleans,Unknown,United States,2021,TV-MA,Docuseries,<NA>,1,tt15320436,tvSeries,Jailbirds New Orleans,0,2021,6.5,337


In [32]:
netflix_imdb_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17781 entries, 0 to 17780
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   type              17781 non-null  object 
 1   title             17781 non-null  object 
 2   director          17781 non-null  object 
 3   country           17781 non-null  object 
 4   release_year      17781 non-null  int64  
 5   rating            17781 non-null  object 
 6   listed_in         17781 non-null  object 
 7   Movie_duration    14173 non-null  Int64  
 8   TV Show_duration  3608 non-null   Int64  
 9   tconst            17781 non-null  object 
 10  titleType         17781 non-null  object 
 11  primaryTitle      17781 non-null  object 
 12  isAdult           17781 non-null  int64  
 13  startYear         17781 non-null  Int64  
 14  averageRating     17781 non-null  float64
 15  numVotes          17781 non-null  int64  
dtypes: Int64(3), float64(1), int64(3), objec

In [33]:
netflix_imdb_df=netflix_imdb_df.drop(columns=['titleType','primaryTitle','startYear'])

In [34]:
netflix_imdb_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17781 entries, 0 to 17780
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   type              17781 non-null  object 
 1   title             17781 non-null  object 
 2   director          17781 non-null  object 
 3   country           17781 non-null  object 
 4   release_year      17781 non-null  int64  
 5   rating            17781 non-null  object 
 6   listed_in         17781 non-null  object 
 7   Movie_duration    14173 non-null  Int64  
 8   TV Show_duration  3608 non-null   Int64  
 9   tconst            17781 non-null  object 
 10  isAdult           17781 non-null  int64  
 11  averageRating     17781 non-null  float64
 12  numVotes          17781 non-null  int64  
dtypes: Int64(2), float64(1), int64(3), object(7)
memory usage: 1.9+ MB


In [35]:
netflix_imdb_df.isna().sum()

type                    0
title                   0
director                0
country                 0
release_year            0
rating                  0
listed_in               0
Movie_duration       3608
TV Show_duration    14173
tconst                  0
isAdult                 0
averageRating           0
numVotes                0
dtype: int64

In [38]:
netflix_imdb_df.isna().sum()

type                    0
title                   0
director                0
country                 0
release_year            0
rating                  0
listed_in               0
Movie_duration       3608
TV Show_duration    14173
tconst                  0
isAdult                 0
averageRating           0
numVotes                0
dtype: int64

In [37]:
netflix_imdb_df.to_csv('netflix_imdb.csv')

In [39]:
filtered_df = netflix_imdb_df.copy()

In [41]:
filtered_df.head()

,type,title,director,country,release_year,rating,listed_in,Movie_duration,TV Show_duration,tconst,isAdult,averageRating,numVotes
0,Movie,Dick Johnson Is Dead,Kirsten Johnson,United States,2020,PG-13,Documentaries,90,<NA>,tt11394180,0,7.4,7549
1,TV Show,Ganglands,Julien Leclercq,United States,2021,TV-MA,Crime TV Shows,<NA>,1,tt13278100,0,7.2,4913
2,TV Show,Ganglands,Julien Leclercq,United States,2021,TV-MA,International TV Shows,<NA>,1,tt13278100,0,7.2,4913
3,TV Show,Ganglands,Julien Leclercq,United States,2021,TV-MA,TV Action & Adventure,<NA>,1,tt13278100,0,7.2,4913
4,TV Show,Jailbirds New Orleans,Unknown,United States,2021,TV-MA,Docuseries,<NA>,1,tt15320436,0,6.5,337
